In [1]:
!pip install bertopic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import json
pd.set_option('display.max_colwidth', None)
import plotly.express as px
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import plotly.graph_objs as go


from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 1. Load csv files

In [ ]:
folder_path = '/content/drive/MyDrive/compliance/OPP-115/annotations/'

In [ ]:
column_headers = ['doc_id', 'test_category', 'segment_number', 'unique_policy_identifier', 'label', 'policy_category', 'segment_details', 'policy_date', 'url']

def read_all_policy_files_with_headers(folder_path):
    all_policies = []
    file_count = 0

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            file_count += 1
            print(f"Reading file {file_count}: {file_name}")
            df = pd.read_csv(file_path, header=None)
            df.columns = column_headers
            all_policies.append(df)

    print(f"Total CSV files read: {file_count}")

    combined_df = pd.concat(all_policies, ignore_index=True)

    return combined_df

combined_df = read_all_policy_files_with_headers(folder_path)

print(combined_df.shape)
print(combined_df.columns)


Reading file 1: 744_minecraft.gamepedia.com.csv
Reading file 2: 883_ted.com.csv
Reading file 3: 898_uptodate.com.csv
Reading file 4: 164_adweek.com.csv
Reading file 5: 175_mlb.mlb.com.csv
Reading file 6: 1164_acbj.com.csv
Reading file 7: 1666_wsmv.com.csv
Reading file 8: 1713_latinpost.com.csv
Reading file 9: 1264_citizen.org.csv
Reading file 10: 856_sciencemag.org.csv
Reading file 11: 186_abcnews.com.csv
Reading file 12: 200_washingtonpost.com.csv
Reading file 13: 541_ifsa-butler.org.csv
Reading file 14: 1708_foxsports.com.csv
Reading file 15: 1419_miaminewtimes.com.csv
Reading file 16: 26_nytimes.com.csv
Reading file 17: 559_www.loc.gov.csv
Reading file 18: 1694_lids.com.csv
Reading file 19: 686_military.com.csv
Reading file 20: 1498_ticketmaster.com.csv
Reading file 21: 414_washingtonian.com.csv
Reading file 22: 586_cariboucoffee.com.csv
Reading file 23: 591_google.com.csv
Reading file 24: 1089_freep.com.csv
Reading file 25: 32_voxmedia.com.csv
Reading file 26: 919_uh.edu.csv
Readin

In [ ]:
combined_df.head(1)

,doc_id,test_category,segment_number,unique_policy_identifier,label,policy_category,segment_details,policy_date,url
0,4889,test_category_labeling_highlight,84,3721,0,Other,"{""Other Type"": {""endIndexInSegment"": 1066, ""startIndexInSegment"": 52, ""selectedText"": ""Welcome to Curse! Curse, Inc. & Curse, Ltd. (\""Curse,\"" \""we,\"" \""us\"" or \""our\"") is committed to respecting the privacy rights and concerns of all users of the Curse www.curse.com websites where this Privacy Policy is posted (the \""Site(s)\"") (we refer to the Site(s), the services and software made available through the Site(s), collectively, as the \""Curse Features\""). As such, we have established and implemented this Privacy Policy to inform visitors to the Site(s) and users of the Curse Features how we use and protect the information we collect through the Site(s). By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy. This Privacy Policy is effective as of the date set forth above and is only applicable to the Site(s) and not to any other website that you may be able to access from the Site(s) which may have its own data collection and use practices and policies."", ""value"": ""Introductory/Generic""}}",10/2/12,http://minecraft.gamepedia.com/Minecraft_Wiki:Privacy_policy


### 2. Distribution of policy categories across the dataset

In [ ]:
category_count = combined_df['policy_category'].value_counts()

fig = px.bar(category_count,
             x=category_count.index,
             y=category_count.values,
             labels={'x': 'Policy Category', 'y': 'Count'},
             title='Distribution of Policy Categories',
             width=800,
             height=600)

fig.update_layout(title={'text': 'Distribution of Policy Categories', 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},xaxis_title='Policy Category' )

fig.show()


### 3. Distribution of policy segment length

In [ ]:
def extract_selected_text(segment):
    try:
        segment_dict = json.loads(segment)
        for key, value in segment_dict.items():
            if 'selectedText' in value:
                return value.get("selectedText", "")
        return ""
    except json.JSONDecodeError:
        return ""

combined_df['selectedText'] = combined_df['segment_details'].apply(extract_selected_text)
combined_df['text_length'] = combined_df['selectedText'].apply(len)


In [ ]:
fig = px.box(combined_df, x='policy_category', y='text_length',
             labels={'policy_category': 'Policy Category', 'text_length': 'Text Length'},
             title='Policy Length Distribution Across Policy Categories')
fig.update_layout(title={'x':0.5, 'xanchor': 'center'}, width=800, height=600)
fig.show()


### 4.Top 10 most important words

In [ ]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

combined_df['cleaned_text'] = combined_df['selectedText'].apply(clean_text)

def get_most_common_words(df, category_col, text_col, category, num_words=30):
    category_df = df[df[category_col] == category]
    words = [word for tokens in category_df[text_col] for word in tokens]
    word_counts = Counter(words).most_common(num_words)
    return word_counts

policy_categories = combined_df['policy_category'].unique()

traces = []
buttons = []

for category in policy_categories:
    word_counts = get_most_common_words(combined_df, 'policy_category', 'cleaned_text', category)
    words, counts = zip(*word_counts)

    trace = go.Bar(x=words, y=counts, name=category)
    traces.append(trace)

    buttons.append(dict(
        method='update',
        label=category,
        args=[{'visible': [cat == category for cat in policy_categories]},
              {'title': f'Top 10 Most Common Words in {category} Category'}]
    ))

layout = go.Layout(
    title='Top 30 Most Common Words by Policy Category',
    xaxis={'title': 'Words'},
    yaxis={'title': 'Frequency'},
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
    }]
)

fig = go.Figure(data=traces, layout=layout)

for i, trace in enumerate(fig.data):
    trace.visible = (i == 0)

fig.show()


In [ ]:
from bertopic import BERTopic
import plotly.graph_objects as go
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

def apply_topic_modeling_on_category(data, selected_category):
    category_data = data[data['policy_category'] == selected_category]['selectedText'].values.tolist()

    category_data = [clean_text(segment) for segment in category_data]

    topic_model = BERTopic(language="english")

    topics, probs = topic_model.fit_transform(category_data)

    topic_info = topic_model.get_topic_info()

    top_words = topic_model.get_topic(topic_info.iloc[1].Topic)

    return top_words[:20]

def plot_bert_topic_words(combined_df):
    policy_categories = combined_df['policy_category'].unique()

    initial_category = policy_categories[0]
    top_words = apply_topic_modeling_on_category(combined_df, initial_category)

    words = [word[0] for word in top_words]
    frequencies = [word[1] for word in top_words]

    fig = go.Figure(data=[
        go.Bar(x=words, y=frequencies, marker=dict(color='royalblue'))
    ])

    fig.update_layout(
        title=f"Top 20 Most Common Words in {initial_category} Category",
        xaxis_title="Words",
        yaxis_title="Frequency"
    )

    buttons = []
    for category in policy_categories:
        buttons.append(dict(
            method='update',
            label=category,
            args=[{
                    'x': [[word[0] for word in apply_topic_modeling_on_category(combined_df, category)]],  # Words
                    'y': [[word[1] for word in apply_topic_modeling_on_category(combined_df, category)]]   # Frequencies
                  },
                  {'title': f"Top 20 Most Common Words in {category} Category"}
                 ]
        ))

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=buttons,
                direction="down",
                showactive=True,
                x=0.17,
                xanchor="left",
                pad={'r': 10}
            )
        ]
    )

    fig.show()

plot_bert_topic_words(combined_df)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read polisis and look for overlapping segmentes between OPP and Polisis

In [ ]:
path = '/content/drive/MyDrive/compliance/Union/'
column_headers = ['policy_segment', 'policy_category']

df = pd.read_csv(path + 'train_dataset.csv', header=None, names=column_headers)
print(df.shape)
df.head()

(4153, 2)


,policy_segment,policy_category
0,"Privacy Policy Sci-News.com is committed to protecting and respecting your privacy. To better inform you of our policy concerning user privacy, we have adopted the following terms. Please note that these terms are subject to change, and any such changes will be included on this page.",Policy Change
1,"Privacy Policy Sci-News.com is committed to protecting and respecting your privacy. To better inform you of our policy concerning user privacy, we have adopted the following terms. Please note that these terms are subject to change, and any such changes will be included on this page.",Introductory/Generic
2,"- if you contact us, we may keep a record of that correspondence;",First Party Collection/Use
3,"- if you contact us, we may keep a record of that correspondence;",Data Retention
4,#NAME?,First Party Collection/Use


In [ ]:
opp_segments = combined_df['selectedText'].str.strip().unique()
polisis_segments = df['policy_segment'].str.strip().unique()

overlapping_segments = set(opp_segments).intersection(set(polisis_segments))

print(f"Number of overlapping segments: {len(overlapping_segments)}")
print(overlapping_segments)


Number of overlapping segments: 796
{'ii. Tracking Technologies Set By the Site. With respect to the Tracking Technologies set by the Site, you have a number of options:', 'By interacting with Barnes & Noble in the manner described in this Privacy Policy at any time, you are accepting the practices described in this Privacy Policy and you consent to the application of this Privacy Policy to the collection, storage, use and disclosure of all your personal and other information as described.', 'In addition to this general Policy, each Service may have additional privacy provisions that are specific to the particular Service. These supplemental disclosures are made adjacent to the particular Service at the time we collect the information. These supplemental disclosures also govern the use of your information.', 'Revision Date: March 24th 2015', 'If you have any concern about privacy at IMDb, please send us a thorough description to our help desk, and we will try to resolve it.', 'Please i

In [ ]:
import pandas as pd

opp_segments = combined_df['selectedText'].str.strip().unique()
polisis_segments = df['policy_segment'].str.strip().unique()

overlapping_segments = set(opp_segments).intersection(set(polisis_segments))

overlap_data = []

for index, row in combined_df.iterrows():
    if row['selectedText'].strip() in overlapping_segments:
        overlap_data.append({
            'Policy Segment': row['selectedText'].strip(),
            'Policy Category (OPP)': row['policy_category'],
            'Source': 'OPP-115'
        })

for index, row in df.iterrows():
    if row['policy_segment'].strip() in overlapping_segments:
        overlap_data.append({
            'Policy Segment': row['policy_segment'].strip(),
            'Policy Category (Polisis)': row['policy_category'],
            'Source': 'Polisis'
        })

overlap_df = pd.DataFrame(overlap_data)

print(f"Number of overlapping segments: {len(overlapping_segments)}")
overlap_df.tail()


Number of overlapping segments: 796


,Policy Segment,Policy Category (OPP),Source,Policy Category (Polisis)
2905,"If you are the parent or guardian of a child under the age of thirteen (13), who you believe has provided NOTMC with personally identifiable information through our Online Services, please e-mail us at online@notmc.com so that we can delete the child's information from our records.",NaN,Polisis,"User Access, Edit and Deletion"
2906,"If you are the parent or guardian of a child under the age of thirteen (13), who you believe has provided NOTMC with personally identifiable information through our Online Services, please e-mail us at online@notmc.com so that we can delete the child's information from our records.",NaN,Polisis,International and Specific Audiences
2907,"If you are the parent or guardian of a child under the age of thirteen (13), who you believe has provided NOTMC with personally identifiable information through our Online Services, please e-mail us at online@notmc.com so that we can delete the child's information from our records.",NaN,Polisis,Privacy contact information
2908,"International This English-language Privacy Policy is NOTMC's official statement of our privacy practices. In case of any inconsistency between this English-language Privacy Policy and its translation into any other language, this English-language document will take precedence over such translation.",NaN,Polisis,International and Specific Audiences
2909,"Contact Us If you have any questions regarding our NOTMC's privacy practices, please do not hesitate to contact us as follows: E-mail: online@notmc.com Telephone: (504) 524-4784 U.S. Mail: 2020 St. Charles Ave., 4th Floor, New Orleans 70130",NaN,Polisis,Privacy contact information


In [ ]:
empty_policy_category = df['policy_category'].isna().sum() + (df['policy_category'] == '').sum()

empty_policy_category


0

In [ ]:
opp_nan_percentage = (combined_df['policy_category'].isna().sum() / len(combined_df)) * 100
print(opp_nan_percentage)
print()
polisis_nan_percentage = (df['policy_category'].isna().sum() / len(df)) * 100
print(polisis_nan_percentage)


0.0

0.0


In [ ]:
combined_df['selectedText'] = combined_df['selectedText'].str.strip()
df['policy_segment'] = df['policy_segment'].str.strip()
print(combined_df.duplicated(subset='selectedText').sum())
print(df.duplicated(subset='policy_segment').sum())


12053
1768


In [ ]:
duplicates_combined_df = combined_df[combined_df.duplicated(subset='selectedText', keep=False)]
duplicates_combined_df.shape

(16053, 12)

In [ ]:
duplicates_combined_df.to_csv(path + 'duplicates.csv', index=False)